In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
from transformers import pipeline
from huggingface_hub import login
from diffusers import StableDiffusionPipeline
import gradio as gr
import torch

# Log in to Hugging Face (replace with your token)
login("")

# Load Hugging Face models
speech_to_text = pipeline("automatic-speech-recognition", model="openai/whisper-base")

# Load Stable Diffusion model using diffusers
text_to_image = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16
).to("cuda" if torch.cuda.is_available() else "cpu")

# Speech-to-text function
def transcribe_audio(audio_file):
    try:
        result = speech_to_text(audio_file)
        transcription = result["text"]
        return transcription
    except Exception as e:
        return f"Error in transcription: {str(e)}"

# Text-to-image function
def generate_image_from_text(text):
    try:
        image = text_to_image(text).images[0]  # Generate one image
        return image
    except Exception as e:
        return f"Error in image generation: {str(e)}"

# Combined processing function
def process_audio_and_generate_image(audio_file):
    transcription = transcribe_audio(audio_file)
    if "Error" in transcription:
        return None, transcription

    image = generate_image_from_text(transcription)
    if isinstance(image, str) and "Error" in image:
        return None, image

    return image, transcription

# Gradio interface
iface = gr.Interface(
    fn=process_audio_and_generate_image,
    inputs=gr.Audio(type="filepath", label="Upload audio file (WAV/MP3)"),
    outputs=[
        gr.Image(label="Generated Image"),
        gr.Textbox(label="Transcription")
    ],
    title="Speech-to-Text and Image Generation",
    description="Upload an audio file to transcribe speech to text, and then generate an image based on the transcription.",
)

# Launch the interface
iface.launch(share=True)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa9b6da076b4ade1dc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
